In [2]:
import scanpy
import scipy.sparse as sp
import os
import numpy as np
from scipy.spatial import Delaunay
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

In [3]:
scaleddata=scanpy.read_h5ad('/mnt/xinyi/2021-01-13-mAD-test-dataset/2020-12-27-starmap-mAD-scaled.h5ad')

In [4]:
asavepath=os.path.join('/mnt/xinyi/','starmap','a')

In [7]:
def getA_delaunay(samplename,savepath=None):
    a_size=scaleddata.shape[0]
    a=sp.lil_matrix((a_size,a_size))
    
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==samplename,['x','y']].to_numpy()
    tri = Delaunay(sobj_coord_np)
    
    a_size=sobj_coord_np.shape[0]
    a=sp.lil_matrix((a_size,a_size))
    for tri_i in range(tri.simplices.shape[0]):
        tri_i_idx=tri.simplices[tri_i,:]
        a[tri_i_idx[0],tri_i_idx[1]]=1
        a[tri_i_idx[1],tri_i_idx[0]]=1
        a[tri_i_idx[0],tri_i_idx[2]]=1
        a[tri_i_idx[2],tri_i_idx[0]]=1
        a[tri_i_idx[1],tri_i_idx[2]]=1
        a[tri_i_idx[2],tri_i_idx[1]]=1
    
    a=a.tocsr()
    if savepath !=None:
        sp.save_npz(savepath,a)
    return a

def getA_knn(samplename,k,a_mode,savepath=None):
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==samplename,['x','y']].to_numpy()
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(sobj_coord_np)
    a=nbrs.kneighbors_graph(sobj_coord_np,mode=a_mode)
    if a_mode=='connectivity':
        a=a-sp.identity(sobj_coord_np.shape[0],format='csr')
    if savepath !=None:
        sp.save_npz(savepath,a)
    return a

In [8]:
for samplename in ['AD_mouse9494','AD_mouse9498','AD_mouse9723','AD_mouse9735']:
    getA_delaunay(samplename,os.path.join(asavepath,'a_delaunay_'+samplename+'.npz'))
    getA_knn(samplename,5,'connectivity',os.path.join(asavepath,'knn5_connectivity_'+samplename+'.npz'))
    getA_knn(samplename,5,'distance',os.path.join(asavepath,'knn5_distance_'+samplename+'.npz'))
    getA_knn(samplename,10,'connectivity',os.path.join(asavepath,'knn10_connectivity_'+samplename+'.npz'))
    getA_knn(samplename,10,'distance',os.path.join(asavepath,'knn10_distance_'+samplename+'.npz'))
    getA_knn(samplename,20,'connectivity',os.path.join(asavepath,'knn20_connectivity_'+samplename+'.npz'))
    getA_knn(samplename,20,'distance',os.path.join(asavepath,'knn20_distance_'+samplename+'.npz'))


In [10]:
testa=sp.load_npz(os.path.join(asavepath,'a_delaunay_'+samplename+'.npz'))

In [13]:
np.sum(testa)

48316.0